In [ ]:
! git clone http://github.com/rosettacommons/deepab
%pip install -r deepab/requirements.txt

In [ ]:
import os
import sys
from glob import glob
from tqdm.contrib.concurrent import process_map
import torch
import numpy as np

sys.path.insert(0, "deepab")

OUT_DIR = "predict_structure"
os.makedirs(OUT_DIR, exist_ok=True)

# Notebook setup
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  if not os.path.exists("/content/google_drive/MyDrive/PyRosetta/colab.bin"):
    pyrosettacolabsetup.install_pyrosetta()
  else:
    pyrosettacolabsetup.mount_pyrosetta_install()

import pyrosetta
pyrosetta.init(
    "-mute all -detect_disulf true -detect_disulf_tolerance 1.5 -check_cdr_chainbreaks false"
)

from deepab.predict import load_model, ModelEnsemble, get_cst_defs, build_structure, renumber_pdb
from deepab.annotate_attention import annotate_structure

In [ ]:
! wget https://data.graylab.jhu.edu/ensemble_abresnet_v1.tar.gz
! tar -xf ensemble_abresnet_v1.tar.gz


model_files = [
    "ensemble_abresnet/rs0.pt",
    "ensemble_abresnet/rs1.pt",
    "ensemble_abresnet/rs2.pt",
    "ensemble_abresnet/rs3.pt",
    "ensemble_abresnet/rs4.pt",
]

device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
model = ModelEnsemble(model_files=model_files,
                          load_model=load_model,
                          eval_mode=True,
                          device=device)

In [ ]:
name = "demo_target"

heavy_sequence = 'EVQLVQSGPEVKKPGTSVKVSCKASGFTFMSSAVQWVRQARGQRLEWIGWIVIGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSSISCNDGFDIWGQGTMVTVS' #@param {type:"string"}
light_sequence = 'EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQHYGSSRGWTFGQGTKVEIKRT' #@param {type:"string"}

decoys = 5 #@param {type:"integer"}
single_chain = len(heavy_sequence) == 0 or len(light_sequence) == 0
renumber = True #@param {type:"boolean"}

In [12]:
pred_dir = os.path.join(OUT_DIR, name)
os.makedirs(pred_dir, exist_ok=True)

fasta_file = os.path.join(pred_dir, f"{name}.fasta")
with open(fasta_file, "w") as f:
    f.write(">:H\n{}\n>:L\n{}\n".format(heavy_sequence, heavy_sequence))
cst_defs = get_cst_defs(model, fasta_file, device=device)

if decoys > 0:
    pred_pdb = build_structure(model,
                                fasta_file,
                                cst_defs,
                                pred_dir,
                                target=name,
                                num_decoys=decoys,
                                num_procs=decoys,
                                single_chain=single_chain,
                                device=device)

    if renumber:
        renumber_pdb(pred_pdb, pred_pdb)

In [ ]:
cdr_loop = 'h3' #@param {type:"string"}
cdr_i = cdr_indices(pdb_file, cdr_loop)

attn_pdb_file = pdb_file.replace(".pdb", ".attn.pdb")
os.system(f"cp {pred_pdb} {attn_pdb_file}")

annotate_structure(model, fasta_file, attn_pdb_file, attn_pdb_file, "CA")